In [145]:
import os
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common import action_chains
import time

In [193]:
# local directory settings
main_dir = os.getcwd()
chrome_driver_dir = main_dir+r'\\chromedriver.exe'
match_id_dir = main_dir+r'\\Match ID\\'

# make directory files
try:
    os.chdir(match_id_dir)
    os.chdir(main_dir)
except FileNotFoundError:
    os.mkdir('Match ID')

# data file storing match_id
match_id_csv = match_id_dir+'match_id.csv'
# check if match_id_csv is empty or not
try:
    data = open(match_id_csv, 'r')
    data.close()
except OSError or FileNotFoundError: # match_lineup.csv not made yet, so make one
    data = open(match_id_csv, 'a') # 'a' instead of 'w' to be safe
    # write header
    header = 'league,season,match_id\n'
    data.write(header)
    data.close()


def determine_EPL_match(match_id, chrome_driver_dir=chrome_driver_dir):
    '''(int) -> None
    Given match_id, determine if the match is an EPL match. If so, record league,
    season, and match_id in match_id_csv.
    '''
    # url information
    base_url = r"https://www.whoscored.com"
    match_url = base_url + r"/Matches/" + str(match_id)

    # access match url
    driver = webdriver.Chrome(chrome_driver_dir)
    driver.get(match_url) # go to EPL url

    # see if match is EPL match
    season_tag_soup = BeautifulSoup(driver.page_source, 'lxml')
    season_tag = season_tag_soup.find_all('a')
    for season in season_tag:
        season_text = season.get_text()
        if '                Premier League - ' in season_text:
            league_i, season_i = season_text.replace(' ','').replace('\n','').split('-')
            data = open(match_id_csv, 'a')
            data.write(','.join([league_i, season_i, str(match_id)])+'\n')
            data.close()
            break
    
    # close driver
    driver.delete_all_cookies()
    driver.close()
    
    return None

In [192]:
index_09_10_i = 317783
index_09_10_f = 318590
d_i = 100 # error margin
for i in range(index_09_10_i-d_i, index_09_10_f+d_i+1):
    print(i,)
    t_i = time.time()
    determine_EPL_match(i)
    t_f = time.time()
    print('elapsed time: '+str(round(t_f-t_i,2))+' seconds')
    

317683
elapsed time: 7.94 seconds
317684
elapsed time: 22.34 seconds
317685
elapsed time: 8.15 seconds
317686
elapsed time: 8.34 seconds
317687
elapsed time: 29.45 seconds
317688
elapsed time: 16.08 seconds
317689


WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=77.0.3865.120)


In [149]:
### OLD ###


# local directory settings
main_dir = os.getcwd()
chrome_driver_dir = main_dir+r'\\chromedriver.exe'
match_id_dir = main_dir+r'\\Match ID\\'

# make directory files
try:
    os.chdir(match_id_dir)
    os.chdir(main_dir)
except FileNotFoundError:
    os.mkdir('Match ID')

# data file storing match_id
match_id_csv = match_id_dir+'match_id.csv'


def get_match_id(season_year, chrome_driver_dir=chrome_driver_dir):
    '''(str) -> list
    Given the season_year (ex. '2018/2019'), return the match_id list in the season
    and record the match_id in match_id_csv.
    '''

season_year = '2007/2008'
year1, year2 = season_year.split('/')
base_url = r"https://www.whoscored.com"
EPL_url = base_url + r"/Regions/252/Tournaments/2/England-Premier-League"

# access EPL url
driver = webdriver.Chrome(chrome_driver_dir)
driver.get(EPL_url) # go to EPL url

# access url with lineup and substitution information
season_select_soup = BeautifulSoup(driver.page_source, 'lxml')
season_select_option = season_select_soup.find_all('option')
for i in range(len(season_select_option)):
    option_i = str(season_select_option[i])
    if 'England-Premier-League' in option_i and season_year in option_i:
        season_extension = option_i.split('\"')[1]
        break
EPL_season_url = base_url + season_extension
driver.get(EPL_season_url)

# access the calendar tab
calendar_access_soup = BeautifulSoup(driver.page_source, 'lxml')
#button_element = driver.find_element_by_css_selector(".date.button.button-icon-right.ui-state-default")
#driver.execute_script("arguments[0].setAttribute('class','date button button-icon-right ui-state-default ui-state-active')", button_element)
#calendar_element = driver.find_element_by_css_selector(".config-panel")
#driver.execute_script("arguments[0].setAttribute('style','margin-top: 2.25em;')", calendar_element)
button_element = driver.find_element_by_css_selector(".date.button.button-icon-right.ui-state-default")
hov = action_chains.ActionChains(driver).move_to_element(button_element)
time.sleep(6)
hov.click()
hov.perform()

# identify year buttons in the calendar panel
unselected_button = driver.find_elements_by_css_selector("td.selectable")
selected_button = driver.find_elements_by_css_selector("td.selected.selectable")
at_year1 = True # boolean determining if year1 is selected
# find year1 or year2 button from currently unselected buttons
for button in unselected_button:
    if button.get_attribute('data-value') == year1: # year1 not selected
        print('here')
        button.click()
        year1_button = button
        at_year1 = False
        break
    elif button.get_attribute('data-value') == year2: # year2 selected = year1 not selected
        year2_button = button
        at_year1 = True
        break
# find year1 or year2 button from currently selected buttons
for button in selected_button:
    if button.get_attribute('data-value') == year1: # year1 selected
        year1_button = button
        at_year1 = True
        break
    elif button.get_attribute('data-value') == year2: # year2 not selected = year1 selected
        year2_button = button
        at_year1 = False
        break

print(year1_button)
print(year2_button)
print(button)
        
# start from year1
if not at_year1:
    print(button.get_attribute('data-value'))
    print(year1_button.get_attribute('data-value'))
    year1_button.click()        
print('done')

here


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=77.0.3865.120)
